In [29]:
# загружаю нужные библиотеки

import os

import pandas as pd


# прогресс-бар
from tqdm.notebook import trange, tqdm

In [2]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor,

    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)
dates_extractor = DatesExtractor(morph_vocab)
money_extractor = MoneyExtractor(morph_vocab)
addr_extractor = AddrExtractor(morph_vocab)

In [18]:
work = pd.read_excel('C:/Data/Стоматологии/Реестр РосЗдравНадзора/stom_lic_rf_2021_08_22_mos.xlsx', sheet_name='data')

In [19]:
work.head(2)

,name,full_name_licensee,abbreviated_name_licensee,brand_name_licensee,form,address_region,address,ogrn,inn,address_number,...,date_duplicate,termination,date_termination,information,information_regulations,information_suspension_resumption,information_cancellation,information_reissuing,date_reissuing,1
0,Федеральная служба по надзору в сфере здравоох...,Центральный банк Российской Федерации,Банк России,Центральный банк Российской Федерации,Федеральное государственное бюджетное учреждение,Москва,"107016, Россия, г. Москва, Неглинная ул, д. 12",1037700013020,7702235133,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,06.08.2021,1
1,Федеральная служба по надзору в сфере здравоох...,Центральный банк Российской Федерации,Банк России,Центральный банк Российской Федерации,Федеральное государственное бюджетное учреждение,Москва,"107016, Россия, г. Москва, Неглинная ул, д. 12",1037700013020,7702235133,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,06.08.2021,1


In [44]:
data = [] # список для данных

for item in tqdm(work.sub_address): # циклом перибираю адреса
    temp = addr_extractor.find(item)
    
    addr_dict = {} # словарь для элементов адреса
    
    for i in temp.fact.parts: # цикл для составления словарей
        addr_dict[i.type] = i.value
        
    data.append([item, addr_dict])

  0%|          | 0/8797 [00:00<?, ?it/s]

In [45]:
# Записываю в датафрейм

head_list = ['address', 'addr_dict']

work_address = pd.DataFrame(data, columns=head_list)

In [72]:
work_address.to_excel('C:/Data/address_stom_mos.xlsx', encoding='utf8', index=False)

In [73]:
work_address.head(5)

,address,addr_dict
0,"117593, г. Москва, Севастопольский проспект, д...","{'индекс': '117593', 'город': 'Москва', 'просп..."
1,"107016, г. Москва, ул. Петровка, д. 8/11, стр. 1","{'индекс': '107016', 'город': 'Москва', 'дом':..."
2,"117996, г. Москва, ул. Житная, д. 12","{'индекс': '117996', 'город': 'Москва', 'улица..."
3,"125124, г. Москва, ул. Правды, д. 6, стр. 2","{'индекс': '125124', 'город': 'Москва', 'улица..."
4,"125364, г. Москва, ул. Свободы, д. 57, корп. 1","{'индекс': '125364', 'город': 'Москва', 'улица..."


In [109]:
test = '141551, Московская область, г.о. Солнечногорск, р.п. Андреевка, ул. Жилинская, д. 1А'

In [110]:
addr_extractor.find(test)

Match(
    start=0,
    stop=84,
    fact=Addr(
        parts=[AddrPart(
             value='141551',
             type='индекс'
         ),
         AddrPart(
             value='Московская',
             type='область'
         ),
         AddrPart(
             value='Андреевка',
             type='посёлок'
         ),
         AddrPart(
             value='1А',
             type='дом'
         )]
    )
)

In [70]:
work_address['index'] = work_address.addr_dict.apply(lambda x: x['индекс'], axis=1)

TypeError: <lambda>() got an unexpected keyword argument 'axis'

In [71]:
work_address.head()

,address,addr_dict
0,"117593, г. Москва, Севастопольский проспект, д...","{'индекс': '117593', 'город': 'Москва', 'просп..."
1,"107016, г. Москва, ул. Петровка, д. 8/11, стр. 1","{'индекс': '107016', 'город': 'Москва', 'дом':..."
2,"117996, г. Москва, ул. Житная, д. 12","{'индекс': '117996', 'город': 'Москва', 'улица..."
3,"125124, г. Москва, ул. Правды, д. 6, стр. 2","{'индекс': '125124', 'город': 'Москва', 'улица..."
4,"125364, г. Москва, ул. Свободы, д. 57, корп. 1","{'индекс': '125364', 'город': 'Москва', 'улица..."


In [53]:
addr_el = []

for item in work_address.addr_dict:
    addr_el += list(item.keys())    

In [55]:
list(set(addr_el))

['село',
 'корпус',
 None,
 'набережная',
 'площадь',
 'индекс',
 'город',
 'страна',
 'проезд',
 'деревня',
 'переулок',
 'офис',
 'посёлок',
 'улица',
 'бульвар',
 'квартира',
 'дом',
 'проспект',
 'строение',
 'область',
 'шоссе']

In [27]:
str(addr_extractor.find(test))

"Match(start=0, stop=23, fact=Addr(parts=[AddrPart(value='Москва', type='город'), AddrPart(value='Сиреневый', type='бульвар')]))"

In [ ]:
path = 'C:/Data/Prodoctorov/CSV/'

In [ ]:
# вытаскиваю названия файлов в список file_list

file_list = []

for root, dirs, files in os.walk(path):
    
    for filename in files:
        file_list += [filename]  

In [ ]:
file_list

In [ ]:
work = pd.read_csv(path + file_list[0])

for file in file_list[1:]:
    temp = pd.read_csv(path + file)
    work = work.append(temp)

In [ ]:
# удаляю дубликаты

work.drop_duplicates(inplace=True)

In [ ]:
work.shape

In [ ]:
work.to_excel('C:/Data/prodoctorov_mos.xlsx', encoding='utf8', index=False)

In [3]:
path = 'C:/Data/Napopravku/CSV/'

In [4]:
# вытаскиваю названия файлов в список file_list

file_list = []

for root, dirs, files in os.walk(path):
    
    for filename in files:
        file_list += [filename]  

In [5]:
file_list

['napopravku_data_1.csv',
 'napopravku_data_2.csv',
 'napopravku_data_3.csv',
 'napopravku_data_4.csv',
 'napopravku_data_5.csv',
 'napopravku_data_6.csv']

In [6]:
work = pd.read_csv(path + file_list[0])

for file in file_list[1:]:
    temp = pd.read_csv(path + file)
    work = work.append(temp)

In [7]:
work.shape

(1481, 23)

In [8]:
# удаляю дубликаты

work.drop_duplicates(inplace=True)

In [9]:
work.shape

(1481, 23)

In [10]:
work.sample(2)

,Unnamed: 0,clinic_page,clinic,verified,address,metro_station,phone,work_hours,clinic_type,age_profile,...,scores_dict,main_clinic,main_clinic_page,advantages,description,year_f,doc_numbers,specialist,full_price,clinic_table
241,241,https://napopravku.ru/moskva/clinics/stomatolo...,Стоматология «Стелла»,1,"г Москва, ул Куусинена, д 6 к 6",Полежаевская (500 м),+7 (499) 195-13-93,none,Частные специализированные стоматологические к...,Взрослых,...,{},none,none,"['Стоматологическая клиника', 'Специализирован...",none,none,6,"{'Егорова Е. П.': {'rating': 'none', 'review_n...",{'Удаление зуба мудрости': 'Цену уточняйте по ...,{'Тип': 'Частные специализированные стоматолог...
256,256,https://napopravku.ru/moskva/clinics/stomatolo...,Стоматология Мастердент на Волгоградском,1,"г Москва, пр-кт Волгоградский, д 54",Кузьминки (900 м),+7 (499) 685-10-09,none,Частные специализированные стоматологические к...,Взрослых и детей,...,"{'Отлично': '2', 'Хорошо': '0', 'Нормально': '...","Мастердент, сеть стоматологий",https://napopravku.ru/moskva/clinics/masterden...,"['Стоматологическая клиника', 'Специализирован...",none,none,4,{'Дорогина Светлана Николаевна': {'rating': '5...,"{'Брекеты': 'Цену уточняйте по телефону', 'Уда...","{'Головная клиника': 'Мастердент, сеть стомато..."


In [37]:
test = 'Московская обл, г Реутов, пр-кт Юбилейный, д 59'

In [41]:
temp = addr_extractor.find(test)

In [ ]:
work.to_excel('C:/Data/napopravku_mos.xlsx', encoding='utf8', index=False)